In [ ]:
# install all modules to help with data cleaning and storage
!pip install flatdict==4.0.1
!pip install google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery-storage

In [ ]:
# import get and post requests from lib
from requests import get,post
import os
import flatdict
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# documentation - https://aviationstack.com/documentation

In [ ]:
# api keys
# os.environ["API_KEY"]
key="a686bdcebdac602cfbb96ff529bd9cb8"
weather_api_key="0dc74d9415a7e5a964398b39942cf3d9"

In [ ]:
# get info on flights
flights_url = "http://api.aviationstack.com/v1/flights"
airlines_url = "http://api.aviationstack.com/v1/airlines"
# urls for api detailing current weather and historical weather
current_weather_url = "http://api.aviationstack.com/v1/current"
historical_weather_url = "http://api.aviationstack.com/v1/historical"

In [ ]:
# helper function to get data from the apis
def get_from_api(url,query,api_key=key):
    return get(url+"?access_key="+api_key+query).json()

In [ ]:
ba_flights = get_from_api(flights_url, "&airline_iata=BA")
# try getting data current weather in new york
london_weather = get_from_api(current_weather_url, "&query=london", weather_api_key)
london_weather

In [ ]:
# not used right now
# [airline for airline in airlines['data'] if airline['airline_name'] == 'British Airways']

In [ ]:
ba_flights = get_from_api(flights_url, "&airline_iata=BA&offset=100")

In [ ]:
# use Google's inbuilt functions to load in my keys for my account
credentials = service_account.Credentials.from_service_account_file(
    'credentials.json'
)

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client(credentials=credentials)

# send the rows to a bigquery table, depending on the table id provided. if no id is provided it will default to flight data
def send_to_bigquery(rows_to_insert, table_id="data-engineering-416410.flight_data.BA flight info"):

    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if not errors:
        print("New rows have been added.")
    else:
        print("Encountered errors while inserting rows: {}".format(errors))

# Process

In [ ]:
# flatten a deeply nested entry
def flatten(row):
    return dict(flatdict.FlatDict(row, delimiter="__"))

# this will format the times to be used in google big query
def fix_datetime(dictionary):
    dict_copy = dictionary.copy()
    for entry in dict_copy.items():
        if entry[0] in ["departure__scheduled","departure__estimated","departure__actual","departure__estimated_runway","departure__actual_runway","arrival__scheduled","arrival__estimated","arrival__actual","arrival__estimated_runway","arrival__actual_runway", ""]:
            if entry[1] is not None:
                dict_copy[entry[0]] = entry[1][:-6]
    dict_copy['flight_date'] = dict_copy['flight_date']+"T00:00:00"
    # del dict_copy['']
    return dict_copy

In [ ]:
def get_flight_data_and_add_to_bigquery():
    for i in range(0,6000,100):
    # for i in range(0,100,100):
        rows_to_insert = []
        flight_page = get_from_api(flights_url,"&airline_iata=BA&offset="+str(i))
        for item in flight_page['data']:
           rows_to_insert.append(fix_datetime(flatten(item)))
        send_to_bigquery(rows_to_insert)
get_flight_data_and_add_to_bigquery()

In [ ]:
# get data on weather and then put this data into big query
def get_weather_data_and_add_to_bigquery():
    #for i in range(0,6000,100):
    for i in range(0,100,100):
        rows_to_insert = []
        weather_page = get_from_api(current_weather_url, "&query=New York", weather_api_key)
        # flight_page = get_from_api(flights_url,"&airline_iata=BA&offset="+str(i))
        for item in weather_page['data']:
           rows_to_insert.append(fix_datetime(flatten(item)))
        send_to_bigquery(rows_to_insert, table_id='data-engineering-416410.weather')
get_weather_data_and_add_to_bigquery()